# Class 20: Introduction to statistical inference

Plan for today:
- Project topics: dealing with dates
- Review statistical inference
- Continue with hypothesis tests


## Notes on the class Jupyter setup

If you have the *ydata123_2023e* environment set up correctly, you can get the class code using the code below (which presumably you've already done given that you are seeing this notebook).  

In [ ]:
import YData

# YData.download.download_class_code(20)   # get class code    
# YData.download.download_class_code(20, TRUE) # get the code with the answers 

YData.download_data("dow.csv")
YData.download_data("movies.csv")

There are also similar functions to download the homework:

In [ ]:
# YData.download.download_class_file('project_template.ipynb', 'homework')  # downloads the class project template (hopefully you've already done this)

If you are using colabs, you should install polars and the YData packages by uncommenting and running the code below.

In [ ]:
# !pip install https://github.com/emeyers/YData_package/tarball/master

If you are using google colabs, you should also uncomment and run the code below to mount the your google drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import statistics
import pandas as pd
import numpy as np
import plotly.express as px
from urllib.request import urlopen

import matplotlib.pyplot as plt
%matplotlib inline

## 1. Dealing with dates

Often there are columns in a DataFrame that contain information about dates. In order to be able to process this information effectively, it is useful to convert them to "datetime" types. 

Let's start by loading out DOW data. If you look at the dtypes of the columns, you will notice that the Date column is an object type. 


In [ ]:

dow1 = pd.read_csv("dow.csv")
display(dow1.head(3))
print(dow1.dtypes)


To read in the the Date column as a datetime type, we can set the `parse_dates` argument to a list of the columns that have dates. 

In [ ]:
dow2 = pd.read_csv("dow.csv", parse_dates= [0])
display(dow2.head(3))
print(dow2.dtypes)

To only get data in a certain data range, we can create a datetime name and then use it to filter out data to get data in a particular range of dates. 

In [ ]:
import datetime

# Create a datatime object (arguments are year, month and day)

my_date = datetime.datetime(2000, 1, 1)

my_date

In [ ]:
# Create a Series of Booleans indicating which dates are earlier than my_date

dow2["Date"] < my_date


In [ ]:
# Get data before our date

dow3 = dow2.copy()

dow3 = dow3[dow3["Date"] < my_date]

dow3.head(3)

## 2. Review of statistical inference

In statistical inference we use a smaller sample of data to make claims about a larger population of data. 

As an example, let's look at the [2020 election](https://www.cookpolitical.com/2020-national-popular-vote-tracker) between Donald Trump and Joe Biden, and let's focus on the results from the state of Georgia. After all the votes had been counted, the resuts showed that:

- Biden received 2,461,854 votes
- Trump received 2,473,633 votes

Since we have all the votes on election data, we can precisely calculate the population parameter of the proportion of votes that Biden received, which we will denote with the symbol $\pi_{Biden}$. 

Let's create names `num_trump_votes` and `num_biden_votes`, and calculate `true_prop_Biden` which is the value $\pi_{Biden}$. 

In [ ]:
num_trump_votes = 2461854  # 2,461,854
num_biden_votes = 2473633  # 2,473,633

true_prop_Biden = num_biden_votes/(num_biden_votes + num_trump_votes)

true_prop_Biden

The code below creates a DataFrame called `georgia_df` that captures these election results. Each row in the DataFrame represents a votes. The column `Voted Biden` is `True` if a voter voted for Biden and `False` if the voter voted for Trump. 

### Sampling distribution

Suppose 10,000 polls were conducted. How many of them would show that Biden would get the majority of the vote? 

Let's simulate this "sampling distribution" of statistics using the "faster way" we discussed last class.  

To do this we can generate random numbers between 0 and 1. If a number is less than the value of $\pi_{Biden}$, then it is a vote for Biden (i.e., a `True` value) otherwise it is a vote for Trump (`False` value). 

Let's generate one sample of 1,000 voters...


In [ ]:

# usse `np.random.rand()` to generate 1,000 numbers between 0 and 1
thousand_random_nums = np.random.rand(1000)

print(thousand_random_nums[0:5])

# visualize these 1,000 numbers as a histogram
plt.hist(thousand_random_nums, edgecolor = "black");

# add a red vertical line at the value of pi_biden
plt.axvline(true_prop_Biden, color = "red");

In [ ]:
# convert to a vector of Booleans (True = vote for Biden, False = vote for Trump)

voter_sample = thousand_random_nums <= true_prop_Biden

voter_sample[0:5]

In [ ]:
# Calculate the proportion of votes for Biden in our sample

# method 1
sample_biden_prop = np.sum(voter_sample)/1000
print(sample_biden_prop)

# method 2
sample_biden_prop2 = np.mean(voter_sample)
print(sample_biden_prop2)


In [ ]:
# function to generate proportion of Biden voters based on a poll

def generate_prop_biden(poll_size):
    random_sample = np.random.rand(poll_size) <= true_prop_Biden
    return np.mean(random_sample)


In [ ]:
%%time

# sampling distribution of many polls conducted

sample_size = 1000
num_simulations = 10000

sampling_dist = []

for i in range(num_simulations):
    prop_biden = generate_prop_biden(sample_size)
    sampling_dist.append(prop_biden)

In [ ]:
plt.hist(sampling_dist, edgecolor = "black", bins = 10);

## 3. Hypothesis test for a single proportion

In hypothesis testing, we start with a claim about a population parameter (e.g., µ = 4.2, or π = 0.25).

This claim implies we should get a certain distribution of statistics, called "The null distribution". 

If our observed statistic is highly unlikely to come from the null distribution, we reject the claim. 

We can break down the process of running a hypothesis test into 5 steps. 

1. State the null and alternative hypothesis
2. Calculate the observed statistic of interest
3. Create the null distribution 
4. Calculate the p-value 
5. Make a decision

Let's run through these steps now!


#### Step 1: State the null and alternative hypothesis

$H_0: \pi = 0.5$

$H_A: \pi < 0.5$


#### Step 2: Calculate the observed statistic of interest


In [ ]:
# load the data

movies = pd.read_csv("movies.csv")

movies.head(3)

In [ ]:
# reduce data to a smaller number of columns: "title" and "binary"

movies_smaller = movies[["title", "binary"]]

In [ ]:
# calculate the proportion of movies that pass the Bechdel test

booleans_passed = movies_smaller["binary"] == "PASS"

prop_passed = np.mean(booleans_passed)

prop_passed


#### Step 3: Create the null distribution 

We need to create a null distribution, which is the distribution of statistics we would expect to get if the null hypothesis is true. 

**Question**: about what percent of the movies would we expect to pass the Bechdel test if the null distribution was true? 

**Answer**: 50%

Let's create simulated data that is consistent with this!


In [ ]:
# Let's generate one proportion consistent with the null hypothesis

n = movies.shape[0]
print(n)

null_sample = np.random.rand(n) < .5

np.mean(null_sample)


In [ ]:
# Let's write a function to generate a proportions consistent with a null hypothesis

def generate_prop_bechdel(n, null_prop):
    random_sample = np.random.rand(n) <= null_prop
    return np.mean(random_sample)

generate_prop_bechdel(1794, .5)

In [ ]:
# Let's generate a null distribution 

null_dist = []

for i in range(10000):    
    null_dist.append(generate_prop_bechdel(1794, .5))


In [ ]:
# visualize the null distribution 

plt.hist(null_dist, edgecolor = "black", bins = 20) #, range = (.4, .6));
plt.plot(prop_passed, 30, '.', markersize = 30, color = "red");

#### Step 4: Calculate the p-value 

Calculate the proportion of points in the null distribution that are more extreme than the observed statistic. 


In [ ]:
# Calculate the p-value

stats_more_extreme = np.array(null_dist) <= prop_passed

print(stats_more_extreme[0:5])

p_value = np.mean(stats_more_extreme)

p_value

#### Step 5: Make a decision

Since the p-value is very small (essentially zero) it is very unlikely that our statistic come from the null distribution. Thus we will reject the null hypothesis and conclude that less than 50% of movies pass the Bechdel test. 


## 3. Hypothesis test for multiple proportions

In a hypothesis test for multiple proportions, we are testing whether each proportion is equal to a particular value. I.e., we are testing whether $\pi_1 = p_1$, $\pi_2 = p_2$, ..., $\pi_k = p_k$, for some proportions $p_1$, $p_2$, ..., $p_k$.

A special case of this is whether all populations proportions are the same, which can be written as: $\pi_1 = \pi_2 = ... = \pi_k$.


### Movivating example: ALCU vs. Almeda County

As a motivating example, let's look look at a report by the American Civil Liberties Union (ACLU) of Almada County jury selection. In particular, the ACLU claimed that jury panels in Almeda were not representative of the underlying demographics of the population of the citizens who lived there. 

The demographics of Almeda county, and the proportion of people selected to be on jury panels, is shown in the DataFrame below, which is based on 1453 people selected to be on jury panels. Let's use this data to run a hypothesis test to examine whether the proportion of people selected to be on jury panels is consistent with the underlying demographics of Almeda. 


In [ ]:

ethnicities = np.array(['Asian', 'Black', 'Latino', 'White', 'Other'])
population_proportions = np.array([0.15, 0.18, 0.12, 0.54, 0.01])
panel_proportions = np.array([0.26, 0.08, 0.08, 0.54, 0.04])


demographics = pd.DataFrame({"Ethnicity": ethnicities, 
                             "Population proportions": population_proportions, 
                             "Jury proportions": panel_proportions})

display(demographics)

# built in pandas plotting functions
demographics.plot.bar("Ethnicity");
plt.ylabel("Proportion");

### Step 1: State the null and alternative hypotheses

**In words** 

Null hypothesis: The proportions of members on the jury panels of different ethnicities match the underlying demographics. 

Alternative hypothesis: The proportion of at least one ethnicity does not match the underlying demographics. 


**In symbols**

$H_0$: $\pi_{Asian} = .15$,  $\pi_{Black} = .18$,  $\pi_{Latino} = .12$,  $\pi_{White} = .54$,  $\pi_{Other} = .01$

$H_A$: At least one $\pi_{i}$ is different from the values specified in the null hypothesis



### Step 2: Calculate the observed statistic

For our observed statistic we will use the Total Variational Distance (TVD) which is defined as:  $TVD ~ = ~ \sum_{i = 1}^{k} |\pi_i - \hat{p}_i |$

Let's write a function `total_variation_distance(distribution_1, distribution_2)` that can calculate the TVD. We can then use this function to calculate the TVD statistic value for the jurors in Almeda county.


In [ ]:
def total_variation_distance(distribution_1, distribution_2):
    
    return np.sum(np.abs(distribution_1 - distribution_2))


observed_statistic_value = total_variation_distance(panel_proportions, population_proportions)

observed_statistic_value

### Step 3: Create the null distribution 

To create the null distribution we need to simulate drawing random sample proportions from the underlying population.

To do this we can generate (uniform) random numbers between 0 and 1. We can then use the `pd.cut()` function to simulate randomly selected jurors ethnicities and convert these to proportions. 

Once we have these proportions, we can calculate the TVD. If we repreat this process 1,000 times we can get a null distribution. 

In [ ]:
# calculate the cumulative proportions we can use to split the data into categories consistent with the null hypothesis

cumulative_proportions = np.append(0, np.cumsum(population_proportions))

cumulative_proportions


In [ ]:
# generate random jury panelist ethnicities

num_jury_members = 1453

rand_nums = np.random.rand(num_jury_members)

one_sample = pd.cut(rand_nums, cumulative_proportions, labels = ethnicities, ordered = False)

print(one_sample[0:5])

In [ ]:
# get the proportions from our sample

unique, counts = np.unique(one_sample, return_counts=True)

sample_proportions = counts/sum(counts)

sample_proportions

In [ ]:
# Let's convert the following steps into one function

def get_sample_proportions(sample_size, true_proportions):
    
    cumulative_proportions = np.append(0, np.cumsum(true_proportions))
    
    rand_nums = np.random.rand(sample_size)
    one_sample = pd.cut(rand_nums, cumulative_proportions)
    unique, counts = np.unique(one_sample, return_counts=True)
    return counts/sum(counts)

    
get_sample_proportions(1453, population_proportions)



In [ ]:
# Step 3: create null distribution 

null_dist = []

num_null_dist_points = 1000

for i in range(num_null_dist_points):
    
    curr_sample_props = get_sample_proportions(1453, population_proportions)
    curr_tvd = total_variation_distance(curr_sample_props, population_proportions)
    null_dist.append(curr_tvd)

In [ ]:
# plot the null distribution as a histogram

plt.hist(null_dist, edgecolor = "black");


### Step 4: Calculate the p-value

The p-value is the proportion of points in the null distribution that are more extreme than the observed statistic. 


In [ ]:
p_value = np.mean(null_dist >= observed_statistic_value)

p_value

### Step 5: Draw a conclusion

Since the p-value is very small, it is very unlikely our statistic comes from the null distribution. Thus we can reject the null distribution and conclude that the proportion of members of different ethnicities on jury panels in Almeda do not reflect the underlying distribution of ethnicities in the population. 
